In [50]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


`%matplotlib` prevents importing * from pylab and numpy


In [51]:
from pulp import *

![](http://snag.gy/JWfaX.jpg)

In [95]:
# List of all the nodes
Nodes = ["good shape",
         "deteriorating",
         "broken"]
Action = ['maintain','ignore']
rewards = [   
         #maintain ignore
         [1,2],#good
         [1,2],#deteriorate
         [-1,0]#broken
         ]
r = rewards = makeDict([Nodes, Action],rewards,0)
p = probability = {# start     action      end                  probability
         ("good shape", "maintain", "good shape"):       1,
         ("good shape", "ignore",   "good shape"):       0.5, 
         ("good shape", "ignore",   "deteriorating"):    0.5,      
         ("deteriorating", "maintain", "good shape"):    0.9,    
         ("deteriorating", "maintain", "deteriorating"): 0.1,    
         ("deteriorating", "ignore", "deteriorating"):   0.5,    
         ("deteriorating", "ignore", "broken"):          0.5, 
         ("broken", "maintain", "good shape"):           0.2,  
         ("broken", "maintain", "broken"):               0.8,
         ("broken", "ignore", "broken"):                 1}

In [96]:
# declare variables
# Q(node, action)
Q = LpVariable.dicts("Q",(Nodes,Action), lowBound=0)

In [97]:
prob = LpProblem("maximize the long-run expected total reward",LpMaximize)
prob += lpSum([r[v][a]*Q[v][a] for v in Nodes for a in Action])

In [98]:
p[("broken", "ignore", "broken")]

1

In [99]:
#constraint 1:
prob += lpSum([Q[v][a] for v in Nodes for a in Action]) == 1

#constraint 2:
for r in Nodes:
    prob += lpSum([Q[r][a] for a in Action]) - lpSum([p[x,a,v]*Q[v][a] \
                                                      for (x,a,v) in p.keys() if x ==r]) ==0    

In [100]:
prob.writeLP("markov decision chain.lp")

In [101]:
prob.solve()
print "the max long-run expected total reward is %s" %(value(prob.objective))

the max long-run expected total reward is 3571.4286


In [102]:
for v in Nodes:
    for a in Action:
        print 'node(', v,') of action(', a, ') has value of', Q[v][a].varValue

node( good shape ) of action( maintain ) has value of 0.0
node( good shape ) of action( ignore ) has value of 0.0
node( deteriorating ) of action( maintain ) has value of 0.35714286
node( deteriorating ) of action( ignore ) has value of 0.0
node( broken ) of action( maintain ) has value of 0.0
node( broken ) of action( ignore ) has value of 0.64285714


In [103]:
prob

maximize the long-run expected total reward:
MAXIMIZE
-1*Q_broken_maintain + 2*Q_deteriorating_ignore + 10000*Q_deteriorating_maintain + -1*Q_good_shape_ignore + 5*Q_good_shape_maintain + 0
SUBJECT TO
_C1: Q_broken_ignore + Q_broken_maintain + Q_deteriorating_ignore
 + Q_deteriorating_maintain + Q_good_shape_ignore + Q_good_shape_maintain = 1

_C2: - 0.5 Q_deteriorating_ignore + 0.5 Q_good_shape_ignore
 + 0 Q_good_shape_maintain = 0

_C3: - 0.5 Q_broken_ignore + 0.5 Q_deteriorating_ignore
 + 0.9 Q_deteriorating_maintain - 0.9 Q_good_shape_maintain = 0

_C4: 0 Q_broken_ignore + 0.2 Q_broken_maintain - 0.2 Q_good_shape_maintain = 0

VARIABLES
Q_broken_ignore Continuous
Q_broken_maintain Continuous
Q_deteriorating_ignore Continuous
Q_deteriorating_maintain Continuous
Q_good_shape_ignore Continuous
Q_good_shape_maintain Continuous